In [1]:
%matplotlib inline
import os
import sys
import gc
import time
from tqdm.notebook import tqdm

import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import plot_model

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from commons.configuration_manager import ConfigurationManager
from src.utilities.transformer import Transformer
from src.learning.training.generator import Generator, GenFiles
from src.learning.models import create_cnn, create_mlp, create_multi_model, create_cnn_2, create_cnn_alone, create_mlp_2, create_multi_model_2, create_multi_model_3

In [2]:
def plot_stuff(title, plot_elems, figsize=(18, 10)):
    fig=plt.figure(figsize=figsize)
    plt.title(title)
    #plt.ylabel('dunno')
    plt.xlabel('Epoch')
    x = np.arange(0, len(plot_elems[0]['data']), 1)
    
    for plot_elem in plot_elems:
        plt.errorbar(x, plot_elem['data'], yerr=plot_elem['error'], label=plot_elem['label'], alpha=plot_elem['alpha'], fmt='-o', capsize=5)

    plt.grid(axis='x')
    plt.legend(loc='best', prop={'size': 15})
    plt.show()
    plt.savefig('./' + title + '.png')

In [3]:
config_manager = ConfigurationManager()
config = config_manager.config

memory_variants = [(1, 1), (4, 1), (4, 4)]
memory = memory_variants[0]

model_path = '../../training/models/'

In [4]:
# Model experiments

epochs = 10
batch_size = 32
verbose = 1

losses = []
val_losses = []

generator = Generator(config, memory_tuple=memory, base_path='../../training/', batch_size=batch_size, column_mode='steer', separate_files=True)
frame_shape, numeric_shape, diff_shape = generator.get_shapes()
print(frame_shape)
print(numeric_shape)
print(diff_shape)

models = []

mlp = create_mlp_2(input_shape=numeric_shape)
cnn = create_cnn_2(input_shape=frame_shape)
models.append((create_multi_model_2(mlp, cnn, output_shape=diff_shape), generator.generate_with_numeric))

models.append((create_cnn_alone(input_shape=frame_shape, output_shape=diff_shape), generator.generate))

for model, generate_method in tqdm(models):
    result_desc = 'n{}_m{}'.format(*memory)
    tqdm.write(result_desc)

    hist = model.fit(generate_method(data='train'),
                     steps_per_epoch=generator.train_batch_count,
                     validation_data=generate_method(data='test'),
                     validation_steps=generator.test_batch_count,
                     epochs=epochs, verbose=verbose)

    model_file_prefix = 'model_n{}_m{}'.format(*memory)
    model_file_suffix = '_{}.{}'

    count = len([fn for fn in os.listdir(model_path) if fn.startswith(model_file_prefix) and fn.endswith('.h5')])
    plot_model(model, to_file=model_path + model_file_prefix + model_file_suffix.format(count + 1, 'png'), show_shapes=True)
    model.save(model_path + model_file_prefix + model_file_suffix.format(count + 1, 'h5'))
    
    current_loss = hist.history['loss']
    current_val_loss = hist.history['val_loss'] 
    
    losses.append(current_loss)
    val_losses.append(current_val_loss)
    
    tqdm.write("Loss per epoch: {}".format(current_loss))
    tqdm.write("Validation loss per epoch: {}".format(current_val_loss))
    
    gc.collect()

(60, 180, 3)
(1,)
1


n1_m1
Train for 5455 steps, validate for 962 steps
Epoch 1/10
5455/5455 [==============================] - 405s 74ms/step - loss: 0.2888 - val_loss: 0.2055
Epoch 2/10
5455/5455 [==============================] - 417s 76ms/step - loss: 0.2087 - val_loss: 0.1901
Epoch 3/10
5455/5455 [==============================] - 415s 76ms/step - loss: 0.1971 - val_loss: 0.1849
Epoch 4/10
5455/5455 [==============================] - 414s 76ms/step - loss: 0.1911 - val_loss: 0.1819
Epoch 5/10
5455/5455 [==============================] - 413s 76ms/step - loss: 0.1880 - val_loss: 0.1860
Epoch 6/10
5455/5455 [==============================] - 413s 76ms/step - loss: 0.1855 - val_loss: 0.1829
Epoch 7/10
5455/5455 [==============================] - 413s 76ms/step - loss: 0.1841 - val_loss: 0.1800
Epoch 8/10
5455/5455 [==============================] - 415s 76ms/step - loss: 0.1826 - val_loss: 0.1831
Epoch 9/10
5455/5455 [==============================] - 417s 76ms/step - loss: 0.1820 - val_loss: 0.1796
Epoc

In [ ]:
# Memory experiments

epochs = 10
batch_size = 64
verbose = 1

losses = []
val_losses = []
    
for memory in tqdm(memory_variants):
    result_desc = 'n{}_m{}'.format(*memory)
    tqdm.write(result_desc)
    
    generator = Generator(config, memory_tuple=memory, base_path='../../training/', batch_size=batch_size, column_mode='steer', separate_files=True)
    frame_shape, numeric_shape, diff_shape = generator.get_shapes()

    mlp = create_mlp_2(input_shape=numeric_shape)
    cnn = create_cnn_2(input_shape=frame_shape)
    multi = create_multi_model_2(mlp, cnn, output_shape=diff_shape)

    hist = multi.fit(generator.generate(data='train'),
                     steps_per_epoch=generator.train_batch_count,
                     validation_data=generator.generate(data='test'),
                     validation_steps=generator.test_batch_count,
                     epochs=epochs, verbose=verbose)

    model_file_prefix = 'model_n{}_m{}'.format(*memory)
    model_file_suffix = '_{}.{}'

    count = len([fn for fn in os.listdir(model_path) if fn.startswith(model_file_prefix) and fn.endswith('.h5')])
    plot_model(multi, to_file=model_path + model_file_prefix + model_file_suffix.format(count + 1, 'png'), show_shapes=True)
    multi.save(model_path + model_file_prefix + model_file_suffix.format(count + 1, 'h5'))
    
    current_loss = hist.history['loss']
    current_val_loss = hist.history['val_loss'] 
    
    losses.append(current_loss)
    val_losses.append(current_val_loss)
    
    tqdm.write("Loss per epoch: {}".format(current_loss))
    tqdm.write("Validation loss per epoch: {}".format(current_val_loss))
    
    gc.collect()

In [ ]:
loss_data = []
val_loss_data = []

for i in range(0, len(losses)):
    val_loss_data.append({'data': val_losses[i], 'label': key, 'alpha': 1.0})
    loss_data.append({'data': losses[i], 'label': key, 'alpha': 1.0})
        
plot_stuff("val losses", val_loss_data, figsize=(10, 14))
plot_stuff("losses", loss_data, figsize=(10, 14))

In [ ]:
import tensorflow as tf

generator = Generator(config, memory_tuple=memory, base_path='../../training/', batch_size=32, column_mode='steer', separate_files=True)
frame_shape, numeric_shape, diff_shape = generator.get_shapes()

shapes = ([40, 60, 12], ())
types = (np.float32, np.float32)

train_dataset = tf.data.Dataset.from_generator(generator.generate_single_train, output_types=types, output_shapes=shapes)
train_dataset = train_dataset.batch(batch_size=32)
train_dataset = train_dataset.shuffle(buffer_size=10)

val_dataset = tf.data.Dataset.from_generator(generator.generate_single_test, output_types=types, output_shapes=shapes)
val_dataset = val_dataset.batch(batch_size=32)
val_dataset = val_dataset.shuffle(buffer_size=10)

model = create_cnn_alone(input_shape=frame_shape, output_shape=diff_shape)

hist = model.fit(train_dataset,
                 steps_per_epoch=generator.train_batch_count,
                 validation_data=val_dataset,
                 validation_steps=generator.test_batch_count,
                 epochs=10, verbose=1)